<a href="https://colab.research.google.com/github/Rudra-prasad-tarai/nlpInternship/blob/main/POSTaggerMbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.47.0 datasets seqeval scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=fd834e00a3da9ac92b54c11a70cda4beac535f4821d5e88d84e34c0be761f8e4
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's 

In [2]:
import ast  # safely parse string to Python object
import numpy as np
import transformers
from seqeval.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback  # <-- Add this import
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset, DatasetDict
import time
start = time.time()

2025-05-16 06:50:24.745142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747378225.220240      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747378225.361050      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def extract_tokens_and_tags(example):
    ann = ast.literal_eval(example['Annotated by: Annotator 1 '])  # safely convert string to list of dicts
    tokens = [entry["word"] for entry in ann]
    tags = [entry["entity"] for entry in ann]
    return {"tokens": tokens, "tags": tags}
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2tag[label] for label in sent if label != -100]
        for sent in labels
    ]
    true_predictions = [
        [id2tag[pred] for (pred, label) in zip(sent_preds, sent_labels) if label != -100]
        for sent_preds, sent_labels in zip(predictions, labels)
    ]
    print(classification_report(true_labels,true_predictions,digits = 6))
    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions)
    }
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128

    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs



In [4]:
# Load POS task subset

from datasets import load_dataset
dataset = load_dataset("LingoIITGN/COMI-LINGUA", "POS")

# Convert to tokens and tags
processed_data = dataset.map(extract_tokens_and_tags)

# Keep only necessary columns
processed_data = processed_data.remove_columns([
    "Sentences", "Predicted Tags",
    "Annotated by: Annotator 1 ",
    "Annotated by: Annotator 2",
    "Annotated by: Annotator 3"
])


print(len(processed_data['train']))

# Assuming processed_data is your Dataset/DatasetDict
data = processed_data['train']

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

POS_train.csv:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

POS_test.csv:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15684 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4999 [00:00<?, ? examples/s]

Map:   0%|          | 0/15684 [00:00<?, ? examples/s]

Map:   0%|          | 0/4999 [00:00<?, ? examples/s]

15684


In [5]:
# type(dataset['train'][15683]) # dataset['train'][i] the annotaion by all the annotators for the the sentence

In [6]:
# First split: 80% train, 20% temp
train_test_split = data.train_test_split(
    test_size=0.2,
    seed=42
)

# Second split: 10% val, 10% test
val_test_split = train_test_split['test'].train_test_split(
    test_size=0.5,
    seed=42
)

# Combine
processed_data = DatasetDict({
    'train': train_test_split['train'],
    'validation': val_test_split['train'],
    'test': val_test_split['test']
})

In [7]:
print(len(processed_data['train']))
print(len(processed_data['validation']))
print(len(processed_data['test']))

12547
1568
1569


In [8]:
processed_data['train'][0]

{'tokens': ['नई',
  'दिल्ली',
  ':',
  'देश',
  'की',
  'प्रमुख',
  'दो',
  '-',
  'पहिया',
  'वाहन',
  'निर्माता',
  'कंपनी',
  'टीवीएस',
  'मोटर्स',
  '(',
  'TVS',
  'Motors',
  ')',
  'आज',
  'यानी',
  'सोमवार',
  'को',
  'Apache',
  'RR',
  '310',
  '2021',
  'को',
  'लॉन्च',
  'करने',
  'जा',
  'रही',
  'है',
  '।'],
 'tags': ['PROPN',
  'PROPN',
  'X',
  'NOUN',
  'ADP',
  'ADJ',
  'ADJ',
  'X',
  'ADJ',
  'NOUN',
  'NOUN',
  'NOUN',
  'PROPN',
  'PROPN',
  'X',
  'PROPN',
  'PROPN',
  'X',
  'ADV',
  'CONJ',
  'PROPN',
  'ADP',
  'NOUN',
  'NOUN',
  'NUM',
  'NUM',
  'ADP',
  'VERB',
  'VERB',
  'VERB',
  'VERB',
  'VERB',
  'X']}

In [9]:
# Get all unique tags
unique_tags = list(set(tag for row in data for tag in row['tags']))
tag2id = {tag: i for i, tag in enumerate(sorted(unique_tags))}
id2tag = {i: tag for tag, i in tag2id.items()}
NUM_LABELS = len(tag2id)


In [10]:
print(len(id2tag))

14


In [11]:
# Tokenize

model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=NUM_LABELS,
    id2label=id2tag,
    label2id=tag2id
)

tokenized_train = processed_data['train'].map(tokenize_and_align_labels, batched=True)
tokenized_val = processed_data['validation'].map(tokenize_and_align_labels, batched=True)
tokenized_test = processed_data['test'].map(tokenize_and_align_labels, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/12547 [00:00<?, ? examples/s]

Map:   0%|          | 0/1568 [00:00<?, ? examples/s]

Map:   0%|          | 0/1569 [00:00<?, ? examples/s]

In [12]:
tokenized_train

Dataset({
    features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 12547
})

In [13]:
# Create training arguements
training_args = TrainingArguments(
    output_dir="./mbert-pos-results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
    load_best_model_at_end=True,  # <-- Important for early stopping
    metric_for_best_model="eval_loss",  # <-- Metric to monitor
    greater_is_better=False  # <-- For loss (lower is better)
)

# Create Trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,  # Stop after 3 epochs without improvement
            early_stopping_threshold=0.01  # Minimum improvement to qualify as better
        )
    ],
    compute_metrics=compute_metrics

)


In [14]:
t = trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.295800,0.270807,0.914774,0.865092
2,0.221600,0.239676,0.923511,0.876882
3,0.173400,0.233786,0.929152,0.885597
4,0.150600,0.228765,0.929458,0.885420
5,0.128200,0.240667,0.931052,0.889514


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: Us

              precision    recall  f1-score   support

         ART   0.891626  0.724000  0.799117       250
     ART_NEG   0.945813  0.964824  0.955224       199
          DJ   0.712112  0.750308  0.730711      1622
          DP   0.951636  0.956238  0.953932      4136
          DV   0.800595  0.741728  0.770038      1088
         ERB   0.864468  0.823328  0.843397      3215
          ET   0.845329  0.924623  0.883200       597
         ONJ   0.878866  0.952514  0.914209       716
         OUN   0.810727  0.819374  0.815027      5719
         RON   0.913187  0.901302  0.907205       922
      RON_WH   0.936364  0.927928  0.932127       222
        ROPN   0.817304  0.885362  0.849973      2678
          UM   0.920204  0.945899  0.932874       573
           _   0.952197  0.923313  0.937532      2934

   micro avg   0.862602  0.867597  0.865092     24871
   macro avg   0.874316  0.874339  0.873183     24871
weighted avg   0.863758  0.867597  0.865173     24871



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/

...

              precision    recall  f1-score   support

         ART   0.907895  0.828000  0.866109       250
     ART_NEG   0.960591  0.979899  0.970149       199
          DJ   0.747698  0.750925  0.749308      1622
          DP   0.957875  0.967602  0.962713      4136
          DV   0.817593  0.811581  0.814576      1088
         ERB   0.869931  0.867496  0.868712      3215
          ET   0.868671  0.919598  0.893409       597
         ONJ   0.957627  0.946927  0.952247       716
         OUN   0.835211  0.829516  0.832354      5719
         RON   0.926803  0.933839  0.930308       922
      RON_WH   0.933628  0.950450  0.941964       222
        ROPN   0.867540  0.890217  0.878732      2678
          UM   0.932642  0.942408  0.937500       573
           _   0.956953  0.954669  0.955810      2934

   micro avg   0.884142  0.887057  0.885597     24871
   macro avg   0.895761  0.898081  0.896707     24871
weighted avg   0.884042  0.887057  0.885487     24871



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/

              precision    recall  f1-score   support

         ART   0.900862  0.836000  0.867220       250
     ART_NEG   0.970443  0.989950  0.980100       199
          DJ   0.747045  0.779285  0.762824      1622
          DP   0.954556  0.964942  0.959721      4136
          DV   0.866935  0.790441  0.826923      1088
         ERB   0.873315  0.866252  0.869769      3215
          ET   0.862715  0.926298  0.893376       597
         ONJ   0.938859  0.965084  0.951791       716
         OUN   0.831927  0.827417  0.829666      5719
         RON   0.944009  0.914317  0.928926       922
      RON_WH   0.954751  0.950450  0.952596       222
        ROPN   0.859884  0.886856  0.873162      2678
          UM   0.948854  0.938918  0.943860       573
           _   0.959036  0.949557  0.954273      2934

   micro avg   0.884869  0.885972  0.885420     24871
   macro avg   0.900942  0.898983  0.899586     24871
weighted avg   0.885235  0.885972  0.885416     24871



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/

              precision    recall  f1-score   support

         ART   0.892241  0.828000  0.858921       250
     ART_NEG   0.956311  0.989950  0.972840       199
          DJ   0.758125  0.747842  0.752948      1622
          DP   0.955327  0.966876  0.961067      4136
          DV   0.844444  0.803309  0.823363      1088
         ERB   0.878977  0.876516  0.877745      3215
          ET   0.841867  0.936348  0.886598       597
         ONJ   0.931635  0.970670  0.950752       716
         OUN   0.845228  0.834586  0.839873      5719
         RON   0.941630  0.927332  0.934426       922
      RON_WH   0.954751  0.950450  0.952596       222
        ROPN   0.867843  0.899925  0.883593      2678
          UM   0.943761  0.937173  0.940455       573
           _   0.958791  0.951602  0.955183      2934

   micro avg   0.888835  0.890193  0.889514     24871
   macro avg   0.897924  0.901470  0.899312     24871
weighted avg   0.888607  0.890193  0.889240     24871



In [15]:
metrics = trainer.evaluate(eval_dataset=tokenized_test)
print(metrics)


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: Us

              precision    recall  f1-score   support

         ART   0.815315  0.811659  0.813483       223
     ART_NEG   0.984925  0.956098  0.970297       205
          DJ   0.755768  0.786300  0.770732      1708
          DP   0.965222  0.969496  0.967354      4065
          DV   0.853183  0.813908  0.833083      1021
         ERB   0.882998  0.886510  0.884750      3269
          ET   0.864078  0.903553  0.883375       591
         ONJ   0.937179  0.965654  0.951204       757
         OUN   0.842528  0.835237  0.838867      5778
         RON   0.934125  0.910526  0.922175       950
      RON_WH   0.945545  0.931707  0.938575       205
        ROPN   0.866494  0.888510  0.877364      2637
          UM   0.936909  0.962723  0.949640       617
           _   0.965152  0.948623  0.956816      2978

   micro avg   0.889966  0.891817  0.890891     25004
   macro avg   0.896387  0.897893  0.896980     25004
weighted avg   0.890360  0.891817  0.890992     25004

{'eval_loss': 0.220812141

In [16]:
metrics

{'eval_loss': 0.2208121418952942,
 'eval_accuracy': 0.9318333636970911,
 'eval_f1': 0.8908909308829405,
 'eval_runtime': 10.3788,
 'eval_samples_per_second': 151.174,
 'eval_steps_per_second': 4.818,
 'epoch': 5.0}

In [17]:
end = time.time()
print(f"Time taken : {(end-start)/60} mins")

Time taken : 19.17715177536011 mins
